In [1]:
import json
from openai import OpenAI
import tiktoken

In [2]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8191  # the maximum for text-embedding-3-small is 8191

encoding = tiktoken.get_encoding(embedding_encoding)

In [31]:
with open("../reports.json") as fh:
    data = json.load(fh)

In [23]:
client = OpenAI()

def embed(my_input):
    if len(my_input) > max_tokens:
        my_input = my_input[:max_tokens]
    return client.embeddings.create(
          model=embedding_model,
          input=my_input,
          encoding_format="float"
        )

In [16]:
xs = sorted(reports, key=lambda x: -len(x[1]))

In [36]:
embedded_data = {}
for fn in data:
    embedded_data[fn] = {"reports": [], "metadata": data[fn]["metadata"]}
    for report in data[fn]["reports"]:
        _report = report
        report = " ".join(report).strip()
        tokenized = encoding.encode(report)
        if tokenized:
            embedding = embed(tokenized)
            embedded_data[fn]["reports"].append({"flat_report": report, "lines_report": _report, "tokenized": tokenized, "embedding": embedding})


In [40]:
new_emb_data = {}
for fn in embedded_data:
    new_emb_data[fn] = {"reports": [], "metadata": embedded_data[fn]["metadata"]}
    for report_d in embedded_data[fn]["reports"]:
        report = report_d["flat_report"]
        _report = report_d["lines_report"]
        tokenized = report_d["tokenized"]
        embedding = report_d["embedding"]
        new_emb_data[fn]["reports"].append({"flat_report": report, "lines_report": _report, "tokenized": tokenized, "embedding": embedding.data[0].embedding})

with open("embedded_data.json", "w") as fh:
    json.dump(new_emb_data, fh, indent=4)

In [41]:
import pandas as pd

In [100]:
def dict_to_df(d):
    rows = []
    for fn in d:
        metadata = d[fn]["metadata"]
        for report in d[fn]["reports"]:
            text = report["flat_report"]
            tokenized = report["tokenized"]
            embedding = report["embedding"]
            if len(tokenized) >= 20:
                rows.append((fn, metadata, text, tokenized, embedding))
    df = pd.DataFrame(rows, columns=["file_name", "metadata", "text", "tokenized", "embedding"])
    return df

df = dict_to_df(new_emb_data)

In [101]:
df.text

0         Rapport-Bok och F. - Ä. för 1892 - 8 Mars 1893.
1       No 1. Nikolaus Svensson.   På anmälan af Hustr...
2       No 2. Gustaf Adolf Flodin.   Handlanden Charle...
3       No 3. Laurentia Elisabeth Melander.   Ogifta A...
4       No 4. En assurerad postförsändelse   Egaren af...
                              ...                        
4719    No 192. Olof Gustaf Jonsson Källström.   På an...
4720    No 193. En till sjö¬ mannen Georg Barstad adre...
4721    No 194. Olof Gustaf Jonsson Källström   Åberop...
4722    No 195. Sven Lund¬ qvist,   Bokhållaren D. Bro...
4723    No 196 Karl August Petersson   Kamrern vid fat...
Name: text, Length: 4724, dtype: object

In [70]:
query_embedding = embed("stöld Göteborg").data[0].embedding

In [78]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine

In [108]:
cosine(df.loc[1000].embedding, df.loc[1000].embedding)

0

In [112]:
def search_reports(df, query=None, query_embedding=None, n=3, pprint=True):
    if query_embedding is None:
        if query is None:
            raise Exception("provide either a query-embedding or a query")
        query_embedding = embed(query).data[0].embedding
    df["similarity"] = df.embedding.apply(lambda x: 1 - cosine(x, query_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .text
    )
    if pprint:
        for r in results:
            print(r)
            print()
    return results

In [ ]:
results = search_reports(df, "stöld Stockholm", n=30)

1 Stockholm Polisinspektorn Göteborg Inget ur likt det omtelegraferade stulet hos Petersson

10. Stadsfiskal Cederborg Stockholm. Norell qväll här anhållen innehar 143 kronor nem¬ ligen en 100 kronosedel af Skånes Bank tre tior af Riksbanken, en tia af Christianstads; Sedlarne ej märkte såsom uppgifvits; nekar tillgreppet; fått 70 kronor af sin vid Signesberg boende moder uttagit 70 kronor å Sparbank. Förhållningsorders. Polisinspektorn

13 Stadsfiskalen Cederborg. Stockholm. Är Gravören Petter Stenberg nu i Stock¬ holm. Eljest hvar? Polisinspektorn.

Stadsfiskalen Cederborg. Stockholm Omtelegrafeade Carl Johan Samuelsson torde efterspanas, häktas och hitsändas. Polismästaren

5. Stockholm Polismästaren Göteborg. Borglund enligt telegram i går kl 9.15 e. m. gripen i Nevcastle, dit kommit från Aarhus. Innehar femtusen Kr., placerat resten på ännu okände ställe Polismästaren.

30. Stockholm. Polismästaren Göteborg Vi blifva eder mycket förbundna för några närmare upplysningar dagsposten 

In [120]:
questions = ['Fanns det någon snickare som hette Andersson i Majorna eller Masthugget?',
 'Vilka affärer på Andra långgatan var inblandade i brottsutredningar?',
 'Hur mycket var en pantsatt vinterpaletå värd 1880?',
 'Bodde någon som hette Johansson i Landala?',
 'Vilka brottsmisstänkta lämnade Göteborg med tåg?',
 'För vilka eftersökta personer anges dialekten som signalement?',
 'Vilka hårfärger har eftersökta personer som befaras rymma till Amerika?',
 'Vilken vara var populärast att sälja till pantbank?',
 'Bodde det några prostituerade kvinnor i Haga?',
 'Nämns något idrottsevenemang i polisrapporterna?',
 'I samband med vilka brottstyper skickades fotografier på brottsmisstänkta mellan poliser i Göteborg och Stockholm?',
 'Vad var medelåldern på misstänkta ficktjuvar?',
 'Vilka utländska myndigheter hade Detektiva avdelning kontakt med via telegraf?',
 'Finns det någon inbrottstjuv som blev gripen på bar gärning?',
 'Har någon artist av något slag, typ vaudevilleartist, komiker, akrobat eller damimitatör misstänkts för stöld?',
 'Vilka sorters brott har begåtts, eller misstänkts ha begåtts i Haga?',
 'Har någon från IOGT-logen Heimdall i Göteborg misstänkts ha begått något brott?',
 'Har det skett stölder hos Ungdomslogen Heimdalls?',
 'Vad var det som föranledde att konditor Bräutigam på Östra Hamngatan gillrade en fälla med frätande s k skedvatten?',
 'Var i huset hade ett barnlik gömts på någon av Larmgatorna?',
 'Hur många barnamord registrerades överhuvudtaget i de aktuella volymerna?',
 'Hur långt straff fick den man som i Haga tog ett mått grädde från en bricka och drack upp det?',
 'Vad var det för brott som begicks på restaurang "Automaten"?',
 'Hur många minderåriga flickor anhölls överhuvudtaget?',
 'Fanns det någon pantbank i Landala?',
 'Nämns något om anhållna som så att säga "erbjöds" en Amerika-biljett för att man skulle bli av med dem?',
 'Från Kristina:',
 'Hur ofta skedde stölder av gods vid Skeppsbrokajen?',
 'Hur många stölder skedde på båtarna vid Skeppsbron och Stenpiren?',
 'Vilka brott utsattes emigranter för?',
 'Vilka judiska personer nämns som brottsoffer/kärande?',
 'Vilka ägde de silver/guld-föremål som stals? Var bodde de?',
 'Hur var sömmerskor inblandade i brott (som offer eller aktörer)?',
 'Var någonstans ägde de brott rum som sömmerskor var inblandade i?',
 'Vilka brottstyper var detektivkonstapel Andrén särskilt duktig att klara upp? Hur ändrade det sig med tiden?',
 'Vilka och var, var byggarbetsplatsbrotten på 1880-talet?',
 'Vilka verksamheter ägnade sig dala-kvinnor/dalkullor åt?',
 'Vilka frukter odlades i Göteborg och blev stulna där?']

In [121]:
embedded_questions = [(embed(q), q) for q in questions]

In [123]:
emb_questions = [(e.data[0].embedding, q) for e,q in embedded_questions]

In [125]:
with open("../data/embedded_questions.json", "w") as fh:
    json.dump(emb_questions, fh, indent=4)

In [129]:
for e,q in emb_questions:
    print(f"Question: {q}", end="\n\n")
    results = search_reports(df, query_embedding=e, n=3)
    print("#" * 99)

Question: Fanns det någon snickare som hette Andersson i Majorna eller Masthugget?

No 21 Antonius Vilhelm Andersson   Måndagen den 4 Juli 1898 På grund af Konungens Befall¬ ningshafvandes i Göteborgs och Bohus län resolution af den 15 Juni, hvari¬ genom Poliskammaren anmodats under¬ söka, under hvilken tid och under hvilka omständigheter snickaren Anto¬ nius Vilhelm Andersson vistats i Göte¬ borg, får jag, med hänvisning till bi¬ fogade intyg från Mantalskontoret och samtliga härvarande Pastorsex¬ peditioner, vördsamt meddela, att med anledning af en i handlingarne före¬ kommande uppgift, att Andersson skulle i stor skala hafva drifvit snickeri¬ rörelse här i staden, förfrågan gjorts hos samtligen härvarande äldre utöfvare af snickeriyrket, bland dem Snickare¬ mästarne Samuel Kruse, boende i huset No 14 vid Vallgatan, och Ernst Hoppe,   boende i huset No 7 vid Pusterviks¬ gatan, hvilka drifvit snickerirörelse här i staden, den förre sedan år 1847 och den senare sedan år 1852, och hafv